## Desafio 02. Salários dos funcionários

### Bibliotecas

In [1]:
import pandas as pd

# Módulo de conexão com banco
import sys
sys.path.append('../') 
import funcoesconexao as con

# Desativa warnings
import warnings
warnings.filterwarnings('ignore')

### Carregamento dos Dados

#### Via banco de dados — SQL Server

In [2]:
conexao = con.conectar_bd()

----- Conexão com banco de dados estabelecida com sucesso! -----


In [3]:
# Carregando dataframes
df_departamentos = con.carregar_dados(conexao, 'departamentos')
df_funcionarios  = con.carregar_dados(conexao, 'funcionarios')

# Fechando conexão
con.fechar_conexao_bd(conexao)

In [4]:
# Exibindo dataframes
display(df_departamentos.head())
display(df_funcionarios.head())

,id,nome
0,1,Recursos Humanos
1,2,Marketing
2,3,Engenharia
3,4,Vendas
4,5,Financeiro


,id,nome,sobrenome,salario,departamento_id
0,1,João,Silva,3000,1
1,2,Maria,Souza,4000,2
2,3,Carlos,Oliveira,6500,3
3,4,Ana,Pereira,3200,1
4,5,Pedro,Costa,3900,2


#### Via arquivo .CSV

In [5]:
# Carregando dataframes
df_departamentos = con.carregar_dados_csv('departamentos.csv')
df_funcionarios  = con.carregar_dados_csv('funcionarios.csv')

In [6]:
# Exibindo dataframes
display(df_departamentos.head())
display(df_funcionarios.head())

,id,nome
0,1,Recursos Humanos
1,2,Marketing
2,3,Engenharia
3,4,Vendas
4,5,Financeiro


,id,nome,sobrenome,salario,departamento_id
0,1,João,Silva,3000,1
1,2,Maria,Souza,4000,2
2,3,Carlos,Oliveira,6500,3
3,4,Ana,Pereira,3200,1
4,5,Pedro,Costa,3900,2


### Solução

In [7]:
def salarios_funcionarios(funcionarios: pd.DataFrame, departamentos: pd.DataFrame):
    # Mesclando dataframes
    df_merge = pd.merge(
        funcionarios,
        departamentos,
        left_on='departamento_id',
        right_on='id',
        how='inner',
        suffixes=('_funcionario', '_departamento'),
    )

    # Filtrando funcionários com salários acima de 4000
    df_merge['salario_acima_4000'] = df_merge['salario'] > 4000

    """ 
    - Agrupando por nome do departamento.
    - Calculando a quantidade de funcionários por departamento
    - Calculando a quantidade de funcionários que recebem acima de 4000
    """
    df_agrupado = (
        df_merge.groupby('nome_departamento')
        .agg(
            quantidade_funcionarios=('id_funcionario', 'count'),
            funcionarios_acima_4000=('salario_acima_4000', 'sum'),
        ).reset_index()
    )

    # Filtrando departamentos com pelo menos três funcionários
    df_agrupado = df_agrupado[df_agrupado['quantidade_funcionarios'] >= 3]

    # Calculando a porcentagem de funcionário que recebem acima de 4000 por departamento
    df_agrupado['porcentagem_acima_4000'] = round(
        ((df_agrupado['funcionarios_acima_4000'] / df_agrupado['quantidade_funcionarios']) * 100), 2,
    )

    # Filtrando os três principais
    df_agrupado = df_agrupado.nlargest(3, 'porcentagem_acima_4000')

    # Reposicionando e renomeando colunas do dataframe
    df_agrupado = df_agrupado[
        ['porcentagem_acima_4000', 'nome_departamento', 'quantidade_funcionarios']
    ]
    df_agrupado.columns = [
        'porcentagem_acima_4000',
        'departamento_nome',
        'quantidade_funcionarios',
    ]

    # Ordenando registros pela porcentagem_acima_4000
    df_agrupado.sort_values('porcentagem_acima_4000', ascending=False, inplace=True)

    return df_agrupado

In [8]:
salarios_funcionarios(df_funcionarios, df_departamentos)

,porcentagem_acima_4000,departamento_nome,quantidade_funcionarios
0,100.00,Engenharia,4
1,50.00,Financeiro,4
2,33.33,Marketing,3
